In [5]:
from ortools.linear_solver import pywraplp


def create_data_model():
    """Create the data for the example."""
    data = {}
    weights = [1500,1500,1500,1500,1500,1500,1500,1500,1500,1500,1500,1500,2080,2080,2080,2080,2080,2080,2080,2080,2080,2080,2080,2080,2080]
    values = [100,100,100,100,100,100,100,100,100,100,100,100,150,150,150,150,150,150,150,150,150,150,150,150,150]
    area = [10625]*12+[13125]*13
    data['weights'] = weights
    data['values'] = values
    data['area'] = area
    data['items'] = list(range(len(weights)))
    data['num_items'] = len(weights)
    num_bins = 2
    data['bins'] = list(range(num_bins))
    data['bin_capacities'] = [25000,27600]
    data['bin_floorarea'] = [138650,282705]
    return data



def main():
    data = create_data_model()

    # Create the mip solver with the SCIP backend.
    solver = pywraplp.Solver.CreateSolver('SCIP')

    # Variables
    # x[i, j] = 1 if item i is packed in bin j.
    x = {}
    for i in data['items']:
        for j in data['bins']:
            x[(i, j)] = solver.IntVar(0, 1, 'x_%i_%i' % (i, j))

    # Constraints
    # Each item can be in at most one bin.
    for i in data['items']:
        solver.Add(sum(x[i, j] for j in data['bins']) <= 1)
    # The amount packed in each bin cannot exceed its capacity.
    for j in data['bins']:
        solver.Add(
            sum(x[(i, j)] * data['weights'][i]
                for i in data['items']) <= data['bin_capacities'][j])
    # The number of pallets packed in each bin cannot exceed the floor area
    for j in data['bins']:
        solver.Add(
            sum(x[(i, j)] * data['area'][i]
                for i in data['items']) <= data['bin_floorarea'][j])


    # Objective
    objective = solver.Objective()

    for i in data['items']:
        for j in data['bins']:
            objective.SetCoefficient(x[(i, j)], data['values'][i])
    objective.SetMaximization()

    status = solver.Solve()

    if status == pywraplp.Solver.OPTIMAL:
        print('Total packed value:', objective.Value())
        total_weight = 0
        for j in data['bins']:
            bin_weight = 0
            bin_value = 0
            bin_area = 0
            print('Bin ', j, '\n')
            for i in data['items']:
                if x[i, j].solution_value() > 0:
                    print('Item', i, '- weight:', data['weights'][i], ' value:',
                          data['values'][i])
                    bin_weight += data['weights'][i]
                    bin_value += data['values'][i]
                    bin_area +=data['area'][i]
            print('Packed bin weight:', bin_weight)
            print('Packed bin value:', bin_value)
            print('Packed bin area:',bin_area)
            print()
            total_weight += bin_weight
        print('Total packed weight:', total_weight)
    else:
        print('The problem does not have an optimal solution.')


if __name__ == '__main__':
    main()

Total packed value: 3150.0000000000005
Bin  0 

Item 0 - weight: 1500  value: 100
Item 1 - weight: 1500  value: 100
Item 2 - weight: 1500  value: 100
Item 3 - weight: 1500  value: 100
Item 4 - weight: 1500  value: 100
Item 5 - weight: 1500  value: 100
Item 6 - weight: 1500  value: 100
Item 7 - weight: 1500  value: 100
Item 8 - weight: 1500  value: 100
Item 9 - weight: 1500  value: 100
Item 10 - weight: 1500  value: 100
Item 11 - weight: 1500  value: 100
Packed bin weight: 18000
Packed bin value: 1200
Packed bin area: 127500

Bin  1 

Item 12 - weight: 2080  value: 150
Item 13 - weight: 2080  value: 150
Item 14 - weight: 2080  value: 150
Item 15 - weight: 2080  value: 150
Item 16 - weight: 2080  value: 150
Item 17 - weight: 2080  value: 150
Item 18 - weight: 2080  value: 150
Item 19 - weight: 2080  value: 150
Item 20 - weight: 2080  value: 150
Item 21 - weight: 2080  value: 150
Item 22 - weight: 2080  value: 150
Item 23 - weight: 2080  value: 150
Item 24 - weight: 2080  value: 150
Packe